# Review of practices implementing large messaging

This notebook is to compare the GP2GP performance of two specific practices that have implemented GP2GP 2.2b (allowing for large messaging) on 12th January 2021.

Note when looking at the counts - we are comparing transfers that were initiated in September up until 12th January compared to transfers between 12th January to 17th March (inclusive).

In [1]:
import pandas as pd
import numpy as np
import paths, data
from datetime import datetime

In [2]:
transfer_file_location = "s3://prm-gp2gp-data-sandbox-dev/transfers-sample-3/"
transfer_files = [
    "9-2020-transfers.parquet",
    "10-2020-transfers.parquet",
    "11-2020-transfers.parquet",
    "12-2020-transfers.parquet",
    "1-2021-transfers.parquet",
    "2-2021-transfers.parquet",
    "partial-3-2021-transfers.parquet"
]
transfer_input_files = [transfer_file_location + f for f in transfer_files]
transfers = pd.concat((
    pd.read_parquet(f)
    for f in transfer_input_files
))

In [3]:
asid_lookup_file_location ="s3://prm-gp2gp-data-sandbox-dev/asid-lookup/"
asid_lookup_files = [
    "asidLookup-Dec-2020.csv.gz",
    "asidLookup-Jan-2021.csv.gz",
    "asidLookup-Feb-2021.csv.gz",
    "asidLookup-Mar-2021.csv.gz",
]
asid_lookup_input_files = [asid_lookup_file_location + f for f in asid_lookup_files]
asid_lookups = pd.concat((
    pd.read_csv(f)
    for f in asid_lookup_input_files
))

In [4]:
large_messaging_vision_practices = asid_lookups["NACS"].isin(["K81607", "K81089"])
asids_for_large_messaging_practices = asid_lookups.loc[large_messaging_vision_practices, "ASID"].unique()
asids_for_large_messaging_practices

array(['004730850049', '642159719037'], dtype=object)

In [5]:
transfers['final_error_code'] = transfers['final_error_code'].fillna(0).astype(str)
transfers['sender_error_code'] = transfers['sender_error_code'].fillna(0).astype(str)
transfers['intermediate_error_codes'] = transfers['intermediate_error_codes'].apply(np.array2string)

In [6]:
mid_jan = datetime(year=2021, month=1, day=12)
before_mid_jan = transfers["date_requested"] < mid_jan

is_sending_vision_practice = transfers["sending_practice_asid"].isin(asids_for_large_messaging_practices)
is_requesting_vision_practice = transfers["requesting_practice_asid"].isin(asids_for_large_messaging_practices)
is_sending_and_requesting_vision_practice = is_sending_vision_practice & is_requesting_vision_practice

transfers.loc[:, "role"] = None
transfers.loc[:, "has_lm"] = None


transfers.loc[is_sending_vision_practice, "role"] = "sending vision practice"
transfers.loc[is_requesting_vision_practice, "role"] = "requesting vision practice"
transfers.loc[is_sending_and_requesting_vision_practice, "role"] = "sending and requesting vision practice"
transfers.loc[before_mid_jan, "has_lm"] = False 
transfers.loc[~before_mid_jan, "has_lm"] = True

vision_transfers = transfers[transfers["role"].notna()]
len(vision_transfers)

252

In [7]:
def pivot_by_role_and_lm(df, col):
    return df[["role", "has_lm", col]]\
        .pivot_table(index=["role", "has_lm"], columns=[col], aggfunc=len, fill_value=0)

In [8]:
transfers_counts = pd.concat({
    "total": vision_transfers.groupby(["role", "has_lm"]).size().to_frame('transfers'),
    "status": pivot_by_role_and_lm(vision_transfers, "status"),
    "final error code": pivot_by_role_and_lm(vision_transfers, "final_error_code"),
    "intermediate_error_codes": pivot_by_role_and_lm(vision_transfers, "intermediate_error_codes"),
    "sender error code": pivot_by_role_and_lm(vision_transfers, "sender_error_code")
}, axis=1, join="outer")

transfers_counts

total status             \
                                              transfers FAILED INTEGRATED   
role                                   has_lm                               
requesting vision practice             False         21      1          8   
                                       True          25      0         24   
sending and requesting vision practice True           2      0          2   
sending vision practice                False        129      0         91   
                                       True          75      0         50   

                                                                          \
                                              PENDING PENDING_WITH_ERROR   
role                                   has_lm                              
requesting vision practice             False        1                 11   
                                       True         0                  1   
sending and requesting vision practice True         0                  0   
sending vision practice                False       33                  5   
                                       True        20                  5   

                                              final error code            \
                                                           0.0 11.0 15.0   
role                                   has_lm                              
requesting vision practice             False                20    1    0   
                                       True                 25    0    0   
sending and requesting vision practice True                  2    0    0   
sending vision practice                False               128    0    1   
                                       True                 71    0    4   

                                              intermediate_error_codes  \
                                                                    []   
role                                   has_lm                            
requesting vision practice             False                        21   
                                       True                         25   
sending and requesting vision practice True                          2   
sending vision practice                False                       129   
                                       True                         75   

                                              sender error code            \
                                                            0.0 10.0 14.0   
role                                   has_lm                               
requesting vision practice             False                 10    0   11   
                                       True                  24    0    0   
sending and requesting vision practice True                   2    0    0   
sending vision practice                False                124    0    0   
                                       True                  70    3    0   

                                                             
                                              23.0 30.0 6.0  
role                                   has_lm                
requesting vision practice             False     0    0   0  
                                       True      0    1   0  
sending and requesting vision practice True      0    0   0  
sending vision practice                False     5    0   0  
                                       True      0    0   2

In [9]:
(transfers_counts.transpose() / transfers_counts["total"]["transfers"] * 100).transpose().round(decimals=2)

total status             \
                                              transfers FAILED INTEGRATED   
role                                   has_lm                               
requesting vision practice             False      100.0   4.76      38.10   
                                       True       100.0   0.00      96.00   
sending and requesting vision practice True       100.0   0.00     100.00   
sending vision practice                False      100.0   0.00      70.54   
                                       True       100.0   0.00      66.67   

                                                                          \
                                              PENDING PENDING_WITH_ERROR   
role                                   has_lm                              
requesting vision practice             False     4.76              52.38   
                                       True      0.00               4.00   
sending and requesting vision practice True      0.00               0.00   
sending vision practice                False    25.58               3.88   
                                       True     26.67               6.67   

                                              final error code              \
                                                           0.0  11.0  15.0   
role                                   has_lm                                
requesting vision practice             False             95.24  4.76  0.00   
                                       True             100.00  0.00  0.00   
sending and requesting vision practice True             100.00  0.00  0.00   
sending vision practice                False             99.22  0.00  0.78   
                                       True              94.67  0.00  5.33   

                                              intermediate_error_codes  \
                                                                    []   
role                                   has_lm                            
requesting vision practice             False                     100.0   
                                       True                      100.0   
sending and requesting vision practice True                      100.0   
sending vision practice                False                     100.0   
                                       True                      100.0   

                                              sender error code              \
                                                            0.0 10.0   14.0   
role                                   has_lm                                 
requesting vision practice             False              47.62  0.0  52.38   
                                       True               96.00  0.0   0.00   
sending and requesting vision practice True              100.00  0.0   0.00   
sending vision practice                False              96.12  0.0   0.00   
                                       True               93.33  4.0   0.00   

                                                                
                                               23.0 30.0   6.0  
role                                   has_lm                   
requesting vision practice             False   0.00  0.0  0.00  
                                       True    0.00  4.0  0.00  
sending and requesting vision practice True    0.00  0.0  0.00  
sending vision practice                False   3.88  0.0  0.00  
                                       True    0.00  0.0  2.67